In [2]:
import re
import os
import sys
import numpy as np
import pandas as pd

from functools import reduce
from itertools import chain
from collections import Counter

from tqdm import tqdm
tqdm.pandas()

from pyaspeller import YandexSpeller
from nltk import sent_tokenize
from transliterate import translit
from nltk.tokenize.toktok import ToktokTokenizer
from rnnmorph.predictor import RNNMorphPredictor
from ufal.udpipe import Model, Pipeline, ProcessingError

sys.path.append('../src')
os.environ['CUDA_VISIBLE_DEVICES'] = ""

from encode import *

In [3]:
articles = pd.read_csv('../data/interim/articles.csv')
articles.head()

,id,hr_level_0,hr_level_1,hr_level_2,hr_level_3,hr_level_4,publication,time,title,snippet,text
0,0,0,0,NaN,152.0,NaN,Финмаркет,2018-03-29T14:13:00,В этом году на льготные автокредиты и лизинг б...,FINMARKET.RU - Премьер России Дмитрий Медведев...,В этом году на льготные автокредиты и лизинг б...
1,1,0,0,NaN,152.0,NaN,ТАСС,2018-03-29T14:16:00,Медведев: около 50 тыс. машин продадут в 2018 ...,Около 50 тыс. автомашин будет продано в текуще...,Около 50 тыс. автомашин будет продано в текуще...
2,2,0,0,NaN,152.0,NaN,Российская газета,2018-03-29T14:30:00,Кабмин выделит 7 миллиардов рублей на льготное...,Правительство выделяет 7 миллиардов рублей на ...,Названы ставки по ипотеке и автокредитам в 201...
3,3,0,0,NaN,152.0,NaN,Телеканал 360°,2018-03-29T14:43:00,Правительство РФ выделит порядка 7 млрд на льг...,Из указанной суммы около семи миллиардов напра...,Правительством России предусмотрено свыше 12 м...
4,4,0,0,NaN,152.0,NaN,Интерфакс,2018-03-29T15:38:00,Правительство выделит в 2018 г. около 7 млрд р...,Правительство РФ выделит в 2018 году около 7 м...,Правительство РФ выделит в 2018 году около 7 м...


### Коррекция пунктуации

In [4]:
#заменяем \xad на пробел
cleaner = lambda x: re.sub('\xad', ' ', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

In [5]:
#делаем все дифисы одинаковыми
cleaner = lambda x: re.sub('[—–]', '-', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

In [6]:
#удаляем кавычки
cleaner = lambda x: re.sub('[«»"”“\'’„`‘\*]', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

In [7]:
#вставляем пробелы вокруг слешей, если их не было

cleaner = lambda x: re.sub(' *\/ *', ' / ', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub(' *\\\ *', ' \ ', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub(' *\| *', ' | ', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

#вставляем пробелы вокруг многоточия, если их не было

cleaner = lambda x: re.sub(' *… *', ' … ', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

In [8]:
#удаление скобок

cleaner = lambda x: re.sub('\{[^\{\}]*\}', '', x)

for _ in range(5):

    articles.title = articles.title.apply(cleaner)
    articles.text = articles.text.apply(cleaner)
    
cleaner = lambda x: re.sub('\[[^\[\]]*\]', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub('\]', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub('\([^\(\)]*\)', '', x)

for _ in range(2):
    
    articles.title = articles.title.apply(cleaner)
    articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub('\(', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub('\)', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

cleaner = lambda x: re.sub('\<[^\<\>]*\>', '', x)

articles.title = articles.title.apply(cleaner)
articles.text = articles.text.apply(cleaner)

In [9]:
#Исправление опечаток

speller = YandexSpeller()

changes = lambda x: {change['word']: change['s'][0] for change in speller.spell(x) if change['s']}
speller_func = lambda x: reduce(lambda x, y: x.replace(y[0], y[1]), [x]+list(changes(x).items()))

articles.title = articles.title.progress_apply(speller_func)
articles.text = articles.text.progress_apply(speller_func)

100%|██████████| 11127/11127 [07:27<00:00, 24.85it/s]


### Предобработка новостного корпуса

Предобработка будет включать следующие этапы:

* разбиение текста на предложения
* разбиение предложений на токены
* определение морфологии
* построение синтаксического дерева

#### Разбиение на предложения

Используем nltk.tokenize.sent_tokenize и модель для русского языкак https://github.com/Mottl/ru_punkt

In [10]:
sent_tokenizer = lambda text: sent_tokenize(text, language='russian')

articles['title_sents'] = articles.title.progress_apply(sent_tokenizer)
articles['text_sents'] = articles.text.progress_apply(sent_tokenizer)

articles['title_preproc'] = articles.title_sents
articles['text_preproc'] = articles.text_sents

100%|██████████| 11127/11127 [00:02<00:00, 3878.50it/s]


#### Разбиение на токены

Используем nltk.tokenize.toktok.ToktokTokenizer

In [11]:
toktok = ToktokTokenizer()
word_tokenizer = lambda text: [toktok.tokenize(sent) for sent in text]

articles.title_preproc = articles.title_preproc.progress_apply(word_tokenizer)
articles.text_preproc = articles.text_preproc.progress_apply(word_tokenizer)

100%|██████████| 11127/11127 [00:04<00:00, 2581.21it/s]


#### Морфологический анализ

Хотим воспользоваться RNNMorph, но сталкиваемся с **проблемой**: слова написанные латинскими буквами воспринемаются, как пунктуация.

In [12]:
rnnmorph = RNNMorphPredictor()
morph_predictor = rnnmorph.predict_sentences

#Например Hyundai Tucson воспринемается как пунктуация
morph_predictor(articles.loc[9].title_preproc)

[[<normal_form=новый; word=Новый; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Masc|Number=Sing; score=0.9997>,
  <normal_form=кроссовер; word=кроссовер; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9999>,
  <normal_form=hyundai; word=Hyundai; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=tucson; word=Tucson; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=2019; word=2019; pos=NUM; tag=NumForm=Digit; score=1.0000>,
  <normal_form=представленный; word=представлен; pos=ADJ; tag=Degree=Pos|Gender=Masc|Number=Sing|Variant=Short; score=0.9988>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=нью-йорк; word=Нью-Йорке; pos=NOUN; tag=Case=Loc|Gender=Masc|Number=Sing; score=0.9987>]]

Что бы решить данную проблему, запишим все слова используя кирилические символы.

In [13]:
articles_title_translit = []
articles_text_translit = []

for article_index, article in articles.iterrows():
    
    for sent_index, sent in enumerate(article.title_preproc):
        for word_index, word in enumerate(sent):
            
            if re.match(r'[a-zA-Z]+', word):
                articles_title_translit.append((article_index, sent_index, word_index, word))
                article.title_preproc[sent_index][word_index] = translit(word, 'ru')
                
    for sent_index, sent in enumerate(article.text_preproc):
        for word_index, word in enumerate(sent):
            
            if re.match(r'[a-zA-Z]+', word):
                articles_text_translit.append((article_index, sent_index, word_index, word))
                article.text_preproc[sent_index][word_index] = translit(word, 'ru')

morph_predictor(articles.loc[9].title_preproc)

[[<normal_form=новый; word=Новый; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Masc|Number=Sing; score=0.9999>,
  <normal_form=кроссовер; word=кроссовер; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=1.0000>,
  <normal_form=хыундая; word=Хыундаи; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9992>,
  <normal_form=туцсон; word=Туцсон; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9978>,
  <normal_form=2019; word=2019; pos=NUM; tag=NumForm=Digit; score=1.0000>,
  <normal_form=представленный; word=представлен; pos=ADJ; tag=Degree=Pos|Gender=Masc|Number=Sing|Variant=Short; score=0.9998>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=нью-йорк; word=Нью-Йорке; pos=NOUN; tag=Case=Loc|Gender=Masc|Number=Sing; score=0.9985>]]

In [14]:
articles.title_preproc = articles.title_preproc.progress_apply(morph_predictor)
articles.text_preproc = articles.text_preproc.progress_apply(morph_predictor)

100%|██████████| 11127/11127 [21:58<00:00,  8.44it/s]


In [15]:
for article_index, sent_index, word_index, word in articles_title_translit:
    articles.loc[article_index].title_preproc[sent_index][word_index].word = word
    
for article_index, sent_index, word_index, word in articles_text_translit:
    articles.loc[article_index].text_preproc[sent_index][word_index].word = word
    
articles.loc[9].title_preproc

[[<normal_form=новый; word=Новый; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Masc|Number=Sing; score=0.9999>,
  <normal_form=кроссовер; word=кроссовер; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=1.0000>,
  <normal_form=хыундая; word=Hyundai; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9992>,
  <normal_form=туцсон; word=Tucson; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9978>,
  <normal_form=2019; word=2019; pos=NUM; tag=NumForm=Digit; score=1.0000>,
  <normal_form=представленный; word=представлен; pos=ADJ; tag=Degree=Pos|Gender=Masc|Number=Sing|Variant=Short; score=0.9998>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=нью-йорк; word=Нью-Йорке; pos=NOUN; tag=Case=Loc|Gender=Masc|Number=Sing; score=0.9985>]]

In [16]:
articles.title_preproc = articles[['title_sents', 'title_preproc']].apply(lambda x: zip(*x), axis=1)
articles.title_preproc = articles.title_preproc.apply(rnnmorph_encoder).apply(conllu_decoder)
articles = articles.drop(columns=['title_sents'])

articles.text_preproc = articles[['text_sents', 'text_preproc']].apply(lambda x: zip(*x), axis=1)
articles.text_preproc = articles.text_preproc.apply(rnnmorph_encoder).apply(conllu_decoder)
articles = articles.drop(columns=['text_sents'])

print(articles.loc[9].title_preproc)

# sent_id = 1
# text = Новый кроссовер Hyundai Tucson 2019 представлен в Нью-Йорке
1	Новый	новый	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	_	_	_	_
2	кроссовер	кроссовер	NOUN	_	Case=Nom|Gender=Masc|Number=Sing	_	_	_	_
3	Hyundai	хыундая	NOUN	_	Case=Gen|Gender=Fem|Number=Sing	_	_	_	_
4	Tucson	туцсон	NOUN	_	Case=Nom|Gender=Masc|Number=Sing	_	_	_	_
5	2019	2019	NUM	_	NumForm=Digit	_	_	_	_
6	представлен	представленный	ADJ	_	Degree=Pos|Gender=Masc|Number=Sing|Variant=Short	_	_	_	_
7	в	в	ADP	_	_	_	_	_	_
8	Нью-Йорке	нью-йорк	NOUN	_	Case=Loc|Gender=Masc|Number=Sing	_	_	_	SpaceAfter=No




#### Построение синтаксического дерева для каждого предложения

Используем синтаксический парсер UDPipe, который был обучен на корпусе SynTagRus на морфологии RNNMorph

In [17]:
parser_model = Model.load('../models/parser_model.udpipe')
parser_pipeline = Pipeline(parser_model, 'conllu', Pipeline.NONE, Pipeline.DEFAULT, 'conllu')
syntax_parser = lambda x: parser_pipeline.process(x, ProcessingError())

articles.title_preproc = articles.title_preproc.progress_apply(syntax_parser)
articles.text_preproc = articles.text_preproc.progress_apply(syntax_parser)

articles.to_csv('../data/interim/articles_preproc.csv', index=False)

100%|██████████| 11127/11127 [06:31<00:00, 28.39it/s]


In [18]:
print(articles.loc[9].title_preproc)

# sent_id = 1
# text = Новый кроссовер Hyundai Tucson 2019 представлен в Нью-Йорке
1	Новый	новый	ADJ	_	Case=Nom|Degree=Pos|Gender=Masc|Number=Sing	2	amod	_	_
2	кроссовер	кроссовер	NOUN	_	Case=Nom|Gender=Masc|Number=Sing	6	nsubj:pass	_	_
3	Hyundai	хыундая	NOUN	_	Case=Gen|Gender=Fem|Number=Sing	2	nmod	_	_
4	Tucson	туцсон	NOUN	_	Case=Nom|Gender=Masc|Number=Sing	2	appos	_	_
5	2019	2019	NUM	_	NumForm=Digit	4	nummod	_	_
6	представлен	представленный	ADJ	_	Degree=Pos|Gender=Masc|Number=Sing|Variant=Short	0	root	_	_
7	в	в	ADP	_	_	8	case	_	_
8	Нью-Йорке	нью-йорк	NOUN	_	Case=Loc|Gender=Masc|Number=Sing	6	obl	_	SpaceAfter=No




В работе использовалась статья https://habr.com/company/sberbank/blog/418701/